<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/Simple_Unet_Wetlands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode


In [6]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [7]:
directory_path = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData'
os.makedirs(directory_path, exist_ok=True)

In [8]:
# zip_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.zip')
# extract_dir = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData' # destination directory
# for zip_file in zip_files:
#     base_name = os.path.basename(zip_file)[:-4]
#     unzip_dir = os.path.join(extract_dir, base_name)
#     os.makedirs(unzip_dir, exist_ok=True)
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(unzip_dir)

In [9]:
extract_dir = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData'

In [10]:
csv_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.csv')
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [11]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [12]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [13]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [14]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [15]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-15-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [16]:
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15
0             0  0.228634  0.045680  0.003364
1             1  0.341195  0.124668  0.007790


In [17]:
sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']

<ipython-input-17-acfe803626d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
<ipython-input-17-acfe803626d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']


In [18]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,tif_name,subset,13,14,15,random_split,Images_path,Labels_path
0,5121,2049,4097,1025,37,749157.194721,751614.766259,2.322698e+06,2.325155e+06,0.002741,4097_1025.tif,Maui_oneYear_subset,0.002741,0.009592,0.006812,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
3,2049,3073,1025,2049,66,741784.411875,744242.028903,2.320240e+06,2.322698e+06,0.001051,1025_2049.tif,Maui_oneYear_subset,0.000000,0.000000,0.001051,1,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
4,4097,3073,3073,2049,68,746699.579012,749157.145333,2.320240e+06,2.322698e+06,0.096800,3073_2049.tif,Maui_oneYear_subset,0.096800,0.099092,0.007901,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
7,5121,3073,4097,2049,69,749157.176045,751614.789917,2.320240e+06,2.322698e+06,0.046784,4097_2049.tif,Maui_oneYear_subset,0.046784,0.035456,0.019820,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
10,6145,3073,5121,2049,70,751614.801963,754072.422910,2.320240e+06,2.322698e+06,0.082713,5121_2049.tif,Maui_oneYear_subset,0.082713,0.089973,0.017363,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28673,16385,27649,15361,508,805681.972014,808139.563877,2.288291e+06,2.290749e+06,0.034338,27649_15361.tif,Maui_oneYear_subset,0.034338,0.035327,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
269,25601,17409,24577,16385,537,798309.191371,800766.782820,2.285834e+06,2.288291e+06,0.185758,24577_16385.tif,Maui_oneYear_subset,0.185758,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
270,26625,17409,25601,16385,538,800766.792034,803224.352420,2.285834e+06,2.288291e+06,0.462209,25601_16385.tif,Maui_oneYear_subset,0.462209,0.000000,0.000174,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
272,27649,17409,26625,16385,539,803224.350722,805681.983466,2.285834e+06,2.288291e+06,0.003212,26625_16385.tif,Maui_oneYear_subset,0.003212,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...


In [19]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

break


In [20]:
import os
import cv2
import rasterio
import numpy as np
import keras
from keras.utils import to_categorical
from keras import layers, models
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
import glob

In [22]:
def load_and_reshape_image(image_path, img_height, img_width):
    with rasterio.open(image_path) as src:
        image = src.read()
        image = image.transpose((1, 2, 0))
        if image.shape[:2] != (img_height, img_width):
            image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
        if len(image.shape) == 3 and image.shape[2] == 1:  # Check for three dimensions
            image = np.squeeze(image, axis=2)
        return image

def load_images_and_labels(image_files, label_files, img_height, img_width, num_classes):
    images = []
    labels = []

    for image_file in image_files:
        image = load_and_reshape_image(image_file, img_height, img_width)
        image[image <= -3e+38] = np.nan

        if np.any(np.isnan(image)):
            nan_mask = np.isnan(image)
            image[nan_mask] = np.nanmean(image)

        if np.any(np.isinf(image)):
          inf_mask = np.isinf(image)
          image[inf_mask] = np.nanmean(image)

        image = image.astype(np.float32)

        images.append(image)


    for label_file in label_files:
        label = load_and_reshape_image(label_file, img_height, img_width)
        label -= 1
        label = to_categorical(label, num_classes=num_classes)
        labels.append(label)

    return np.array(images), np.array(labels)

class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = 4
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        batch_images, batch_labels = load_images_and_labels(batch_files, self.label_files, self.img_height, self.img_width, self.num_classes)
        return batch_images, batch_labels

KERNEL_SIZE = 512
BANDS = range(7)

def create_model(img_size, num_classes):
    inputs = keras.Input(shape=(img_size[0], img_size[1], len(BANDS)))  # Adjusted for the number of bands

    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)  # Changed to LeakyReLU

    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = layers.LeakyReLU()(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.LeakyReLU()(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])
        previous_block_activation = x

    for filters in [256, 128, 64, 32]:
        x = layers.LeakyReLU()(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.LeakyReLU()(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])
        previous_block_activation = x

    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = create_model((KERNEL_SIZE, KERNEL_SIZE, len(BANDS)), num_classes=25)

# # Load and prepare your data
# image_files = glob.glob(os.path.join('/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset', "*.tif"))
# label_files = glob.glob(os.path.join('/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset', "*.tif"))

# Train the model
training_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==0], sum_df['Labels_path'][sum_df['random_split']==0], 512, 512, 4, 25)
validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 25)


# training_data_generator = DataGenerator(image_files, label_files, KERNEL_SIZE, KERNEL_SIZE, 4, 25)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])
callbacks = [ModelCheckpoint("wetlands.h5", save_best_only=True)]
model.fit(training_data_generator, validation_data_generator, epochs=5, callbacks=callbacks, shuffle=True)


TypeError: ignored